In [1]:
import numpy

In [2]:
import sklearn

In [3]:
import survwrap

In [4]:
X, y = survwrap.load_test_data()
X.shape, y.shape

((198, 84), (198,))

### Generate a (stratified) train-test split and Scale the features (only) 

First do the stratified splitting THEN do scaling, parameterized on X_train set ONLY 

In [5]:
from sklearn.preprocessing import StandardScaler, RobustScaler

In [94]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y,
                                                                            stratify=survwrap.get_indicator(y), 
                                                                           random_state=2308)

In [107]:
scaler = StandardScaler().fit(X_train)
[X_train, X_test] = [ scaler.transform(_) for _ in  [X_train, X_test] ]
X_train.shape, X_test.shape

((148, 84), (50, 84))

In [7]:
#X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

balanced partitioning OK. Robst scaler damages the performance of DSM A LOT.
maybe did something wrong. It is standard scaler for now.

In [108]:
survwrap.get_indicator(y).sum(), survwrap.get_indicator(y_train).sum(), survwrap.get_indicator(y_test).sum(),


(51, 38, 13)

## check possible dimensionality reduction

In [99]:
from sklearn.decomposition import PCA

In [109]:
pca= PCA(n_components=0.995, random_state=2308).fit(X_train)
pca.n_components_

74

Only a modest dimensionality reduction is possible using PCA

In [129]:
## Stratified CV spliter for survival analysis

In [140]:
testkf= RepeatedStratifiedKFold(n_splits=5,n_repeats=2,random_state=2307)
for trn,tst in testkf.split(X_train, survwrap.get_indicator(y_train)):
    print(trn,tst) 

[  1   2   3   6   7   8   9  10  11  13  14  16  17  19  20  22  24  25
  26  28  29  30  31  32  33  34  35  36  37  39  41  42  43  45  46  47
  48  49  51  52  53  54  55  57  59  60  61  62  63  67  68  70  71  72
  74  76  77  78  79  80  82  83  84  85  86  87  88  89  90  91  92  93
  95  96  97  98  99 101 102 104 105 106 108 109 110 111 112 114 115 116
 117 118 119 120 121 122 123 125 126 127 128 129 130 131 132 133 135 136
 137 138 139 140 141 142 144 145 146 147] [  0   4   5  12  15  18  21  23  27  38  40  44  50  56  58  64  65  66
  69  73  75  81  94 100 103 107 113 124 134 143]
[  0   1   3   4   5   7   8  10  11  12  13  15  16  17  18  19  20  21
  22  23  24  25  26  27  28  29  30  31  32  33  34  35  38  39  40  42
  43  44  45  46  47  49  50  51  54  55  56  57  58  61  63  64  65  66
  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84
  85  87  88  89  90  91  94  95  97  98 100 101 103 104 106 107 109 110
 112 113 114 115 116 117 119 120

In [130]:
from sklearn.model_selection import RepeatedKFold, StratifiedKFold

# test coxnet

In [110]:
coxnet = survwrap.CoxNet(rng_seed=2308)
coxnet.fit(X_train, y_train)

CoxNet(rng_seed=2308, l1_ratio=0.5, verbose=False, fit_baseline_model=False)

In [111]:
coxnet.score(X_test, y_test)

0.5330578512396694

In [48]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [16]:
def optimize(estimator, X, y, mode='sklearn-grid', user_grid=[] , cv=None):
    if mode == 'sklearn-grid':
        if not user_grid: 
            user_grid = estimator.get_parameter_grid()
        gs = GridSearchCV(estimator, user_grid, refit=True, cv=cv, n_jobs=3, random)
        gs.fit(X, y)
        return gs.best_estimator_, gs.best_params_, gs
    raise ValueError(f'unknown mode parameter: "{mode}"')

In [103]:
survwrap.CoxNet().get_parameter_grid()

{'l1_ratio': [0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99]}

In [171]:
opt_coxnet, opt_coxnet_params, opt_coxnet_search = optimize(survwrap.CoxNet(), X_train, y_train, user_grid={'l1_ratio':[0.01,0.25,0.5,0.75,0.8,0.99]},
                                                           cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=2308).split(X_train,survwrap.get_indicator(y_train)))
opt_coxnet.score(X_test, y_test), opt_coxnet_params

(0.5764462809917356, {'l1_ratio': 0.01})

In [113]:
def test_model(model_constructor, **model_params):
    m = model_constructor(**model_params, rng_seed=2308)
    m.fit(X_train, y_train)
    o = m.score(X_train, y_train)
    s = m.score(X_test, y_test)
    print(f'Train: {o: .4f} , Test: {s:.4f}')
    return s

# Test DSM

In [114]:
grid =survwrap.DeepSurvivalMachines.get_parameter_grid()
grid

{'n_distr': [2],
 'distr_kind': ['Weibull'],
 'batch_size': [16],
 'layer_sizes': [[10]],
 'learning_rate': [0.001],
 'validation_size': [0.1],
 'max_epochs': [10],
 'elbo': [False]}

In [115]:
test_model(survwrap.DeepSurvivalMachines, batch_size=16, layer_sizes=[16], n_distr=2, max_epochs=200)

  3%|█▋                                                        | 6/200 [00:00<00:09, 19.53it/s]


Train:  0.7693 , Test: 0.6302


0.6301652892561983

In [116]:
test_model(survwrap.DeepSurvivalMachines, batch_size=16, layer_sizes=[7,7,7], n_distr=2, max_epochs=200)

 10%|█████▍                                                   | 19/200 [00:00<00:05, 35.97it/s]


Train:  0.8681 , Test: 0.6467


0.6466942148760331

In [121]:
test_model(survwrap.DeepSurvivalMachines, batch_size=16, layer_sizes=[5]*5, n_distr=2, max_epochs=200)

  9%|█████▏                                                   | 18/200 [00:00<00:05, 34.87it/s]

Train:  0.7767 , Test: 0.6085


0.6084710743801653

In [128]:
test_model(survwrap.DeepSurvivalMachines,layer_sizes=[41], max_epochs=200)

  3%|█▋                                                        | 6/200 [00:00<00:08, 22.83it/s]

Train:  0.7555 , Test: 0.5971


0.5971074380165289

In [165]:
grid['n_distr']=[1]
some_primes = [3,5,7,11,13,17,19,23,29,31,37]
grid['layer_sizes']=[ [_] for _ in some_primes] + [ [_]*3 for _ in some_primes]
    #[31]*2,[29]*2,[27]*2, [25]*2, [23]*2, [21]*2, [19]*2, [17]*3, [15]*3, 
     #                [12]*4, [10]*5, [8]*6, [7]*7, ]
    #[40],[30,30], [30], [23,23], [13,13,13], [25],[11,11,11],[20],[9,9,9], 
                     #[15],[7,7,7],[10],[5,5,5]]
                     #[16],[14,14], [7,7,7]]
grid['max_epochs']=[200]
grid['batch_size']=[16]
grid

{'n_distr': [1],
 'distr_kind': ['Weibull'],
 'batch_size': [16],
 'layer_sizes': [[3],
  [5],
  [7],
  [11],
  [13],
  [17],
  [19],
  [23],
  [29],
  [31],
  [37],
  [3, 3, 3],
  [5, 5, 5],
  [7, 7, 7],
  [11, 11, 11],
  [13, 13, 13],
  [17, 17, 17],
  [19, 19, 19],
  [23, 23, 23],
  [29, 29, 29],
  [31, 31, 31],
  [37, 37, 37]],
 'learning_rate': [0.001],
 'validation_size': [0.1],
 'max_epochs': [200],
 'elbo': [False]}

In [33]:
from sklearn.model_selection import RepeatedKFold 

In [142]:
# Non stratified CV
#opt_dsm, opt_dsm_params, opt_dsm_search = optimize(survwrap.DeepSurvivalMachines(rng_seed=2308),  X_train, y_train, 
#                                                   user_grid=grid,cv=RepeatedKFold(n_splits=5, n_repeats=3, random_state=2308))
#opt_dsm.score(X_test, y_test), opt_dsm_params

In [166]:
# Stratified CV
opt_dsm, opt_dsm_params, opt_dsm_search = optimize(survwrap.DeepSurvivalMachines(rng_seed=2308),  X_train, y_train, 
                                                   user_grid=grid,cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=2308).split(X_train,survwrap.get_indicator(y_train)))
opt_dsm.score(X_test, y_test), opt_dsm_params

  2%|█▏                                                        | 4/200 [00:00<00:12, 15.63it/s]


(0.6590909090909091,
 {'batch_size': 16,
  'distr_kind': 'Weibull',
  'elbo': False,
  'layer_sizes': [31, 31, 31],
  'learning_rate': 0.001,
  'max_epochs': 200,
  'n_distr': 1,
  'validation_size': 0.1})

In [167]:
cv_res = opt_dsm_search.cv_results_
rez=sorted(zip(cv_res['rank_test_score'], cv_res['mean_test_score'] , cv_res['std_test_score'], 
               cv_res['mean_test_score'] - cv_res['std_test_score'], opt_dsm_search.cv_results_['params']))

In [168]:
rez[:10]

[(1,
  0.6708545595719525,
  0.07941291489368205,
  0.5914416446782704,
  {'batch_size': 16,
   'distr_kind': 'Weibull',
   'elbo': False,
   'layer_sizes': [31, 31, 31],
   'learning_rate': 0.001,
   'max_epochs': 200,
   'n_distr': 1,
   'validation_size': 0.1}),
 (2,
  0.6660412853776564,
  0.08625703973533061,
  0.5797842456423258,
  {'batch_size': 16,
   'distr_kind': 'Weibull',
   'elbo': False,
   'layer_sizes': [13],
   'learning_rate': 0.001,
   'max_epochs': 200,
   'n_distr': 1,
   'validation_size': 0.1}),
 (3,
  0.6580985055372891,
  0.039886136922990534,
  0.6182123686142986,
  {'batch_size': 16,
   'distr_kind': 'Weibull',
   'elbo': False,
   'layer_sizes': [23, 23, 23],
   'learning_rate': 0.001,
   'max_epochs': 200,
   'n_distr': 1,
   'validation_size': 0.1}),
 (4,
  0.6544769350353211,
  0.09844260219660868,
  0.5560343328387124,
  {'batch_size': 16,
   'distr_kind': 'Weibull',
   'elbo': False,
   'layer_sizes': [23],
   'learning_rate': 0.001,
   'max_epochs': 20

In [169]:
low_bound = sorted(rez[:10], key= lambda k: k[3], reverse=True)
for lb in [ _ for _ in low_bound if _[3] > 0.5] :
    print(lb[0:4])
    test_model(survwrap.DeepSurvivalMachines, **lb[4])

(3, 0.6580985055372891, 0.039886136922990534, 0.6182123686142986)


  2%|█▍                                                        | 5/200 [00:00<00:10, 18.33it/s]


Train:  0.7992 , Test: 0.6632
(1, 0.6708545595719525, 0.07941291489368205, 0.5914416446782704)


  2%|█▏                                                        | 4/200 [00:00<00:13, 14.08it/s]


Train:  0.8132 , Test: 0.6591
(5, 0.6502447495437347, 0.05905006532574704, 0.5911946842179876)


...

Train:  0.8392 , Test: 0.6178
(6, 0.6369849876644174, 0.10775570949941397, 0.5292292781650034)


  4%|██                                                        | 7/200 [00:00<00:06, 27.71it/s]


Train:  0.7824 , Test: 0.6467
(9, 0.6261967010773682, 0.10381205537523215, 0.522384645702136)


  2%|▊                                                         | 3/200 [00:00<00:15, 12.76it/s]

Train:  0.6570 , Test: 0.6694


In [164]:
test_model(survwrap.DeepSurvivalMachines, batch_size=16, layer_sizes=[13], n_distr=1, max_epochs=200)

  4%|██▎                                                       | 8/200 [00:00<00:07, 27.00it/s]

Train:  0.8348 , Test: 0.6612


0.6611570247933884

# Appunti di varie prove DSM

## Insieme di prova (Breast cancer)

Piccolo insieme bastardo. 200 elementi, 84 features e solo il 25% di eventi non censurati.
Coxnet di default va male (c-index 53%), DSM richiede cura per inziare a generalizzare.

## Stabilità risultato

1. stratificare, stratificare rispetto all'evento. Usando insiemi stratificati rispetto all'evento DSM inizia a funzionare in maniera meno erratica.
2. Se si stratifica anche la CV, L'ottimizzazione DSM tende a fornire risultati buoni per tutta le soluzioni top e anche CoxNet ci guadagna in ottimizzazione (c-index: 57%).
   potrebbe sembrare ovvio a posteriori, dato che stiamo imponendo un "contenuto di informazione" simile tra train e test. ma non mi pare una forzatura clamorosa.

# Ottimizzazione parametri DSM

1. Quello più difficile da gestire è ovviamente la topologia della rete. Usando un solo strato nascosto si vede che anche cambiando di un solo valore la dimensione si può passare da una rete che generalizza a una che fa schifo, quindi bisognerebbe fare uno scan senza salti. O fare una random search.
   Alla fine mi sono inventato un protocollo scemo ma che sembra andare ragionevolmente (su breast cancer). Fare uno scan per i valori de numeri primi compresi tra 3 e n_features/2 (olè!)
2. Reti a "parametri equivalenti", (la procedura del quant): fatti tanti test, c'è un fenomeno curioso: le reti con numero pari di strarti nascosti generalizzano male. Mah!
   Quindi alla fine ho testato sistematicamente solo quelle con strati dispari (1,3,5). Ma per quelle con 5 iniziano ad apparire problemi di stabilità numerica, probabilmente quando sono troppo grandi.
3. numero di funzioni di base (Weibull): il default è 2, ma si ottengono parecchi buoni risultati anche con una sola. Molte soluzioni con 3 non generalizzano. quindi 1 o 2.
4. Come detto sopra, la stratificazione aiuta molto, così come il fare almeno 10 split (n_splits=5, n_repeats=2) per aver una varianza abbastanza stabile. Sono riuscito a fare griglie con cv stratificate con un trucchino sklearn che implementerei
5. batch size: 10% dell'insieme di train. Sembra OK. mai ottimizzata. così come gli altri parametri legati al learning. Non mi aspetto che incidano.
6. Alla fine della fiera si possono ottenere risulati stabilmente decorosi usando 1 o 3 strati nascosto e 1 sola weibull (test c-index: 65%). Introducendo anche la doppia weibull la griglia diventa più onerosa e possono aumentare le soluzioni solo apparentemente buone ma generalizzano peggio (anche se non disastrose: 58%). Non andare oltre le 2 weibull e 3 strati.
